In [5]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp
import matplotlib.pyplot as plt
import os 
import openpyxl
import numpy as np
from gseapy import barplot, dotplot
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import glob
gset={"data/msigdb.v2023.1.Mm.symbols.gmt"} 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
pip show gseapy

Name: gseapy
Version: 1.1.4
Summary: Gene Set Enrichment Analysis in Python
Home-page: https://github.com/zqfang/gseapy
Author: Zhuoqing Fang
Author-email: fzq518@gmail.com
License: MIT
Location: /research/rgs01/home/clusterHome/rmehmood/.local/lib/python3.10/site-packages
Requires: matplotlib, numpy, pandas, requests, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [7]:
def sheet_info2(filename, outdir):
    # Open the Excel file
    xls = pd.ExcelFile(filename)
    # Get the names of all sheets in the Excel file
    sheet_names = xls.sheet_names
    
    # Iterate over each sheet in the Excel file
    for sheet_name in sheet_names:
        # Read the current sheet into a DataFrame, using the 'gene' column as the index
        df = pd.read_excel(xls, sheet_name, index_col='gene')
        
        # Print the DataFrame for inspection
        print(df)
        # Print the sheet name and the number of rows (genes) in the DataFrame
        print(f"{sheet_name} : {df.shape[0]}")
        print(df)
        
        # Copy the DataFrame to a temporary variable for further processing
        tmp = df
        
        # Sort the DataFrame by the 'avg_log2FC' column in descending order
        tmp = tmp.sort_values('avg_log2FC', ascending=False)
        
        # Extract the ranking information: a DataFrame with only the 'avg_log2FC' column
        rnk = tmp.loc[:, ['avg_log2FC']]
        
        # Check if there are at least 10 genes to perform GSEA; if not, skip this sheet
        if rnk.shape[0] < 10:
            print(f"found few genes so skip GSEA: {sheet_name}")
        else:
            # Create the output directory if it does not exist using a shell command (works in Jupyter/IPython)
            !mkdir -p $outdir
            
            # Save the ranking information to a CSV file
            rnk.to_csv(f"{outdir}/{sheet_name}_rank_{rnk.shape[0]}.csv")
            
            # Run Gene Set Enrichment Analysis (GSEA) using the prerank method from the 'gp' package.
            # Ensure that 'gset' is defined and contains the gene sets.
            pre_res = gp.prerank(rnk=rnk,                # Ranking file with avg_log2FC values
                         gene_sets=list(gset),           # List of gene sets to test against
                         threads=4,                      # Number of threads to use
                         min_size=5,                     # Minimum gene set size to consider
                         max_size=1000,                  # Maximum gene set size to consider
                         permutation_num=1000,           # Number of permutations (adjust to speed up testing)
                         outdir=None,                    # Do not write results to disk automatically
                         seed=6,                         # Random seed for reproducibility
                         verbose=True,                   # Verbose output to track progress
                        )
            
            # Save the detailed GSEA results (res2d) to a CSV file in the output directory
            pd.DataFrame(pre_res.res2d).to_csv(f"{outdir}/GSEA_res2d_out_{sheet_name}.csv")
            
            print("Done work-----------")
            
            # Create a subdirectory for the current sheet's plots
            ofolder = outdir + f"/{sheet_name}"
            !mkdir -p $ofolder
            
            # For each term in the GSEA results with an FDR q-value < 0.05, generate a plot
            for t in pre_res.res2d[pre_res.res2d["FDR q-val"] < 0.05].Term:
                # Generate the enrichment plot for the current term
                axs = pre_res.plot(terms=t)
                # Set the figure size for the plot
                plt.gcf().set_size_inches(10, 7)
                # Save the plot as a PDF file. The term is split using "__" and the second part is used in the filename.
                plt.savefig(f'{ofolder}/{t.split("__")[1]}_plot.pdf', format='pdf')
                # Close the plot to free up memory
                plt.close()


In [ ]:
p53="WT_vs_Muttant_avglfc_0.001minPCT_0.05.xlsx"
result = sheet_info2(filename=p53,outdir=f'GSEA_{p53.split(".xl")[0]}/')

<h1> Read me </h1>

This script processes the output from Seurat's findMarkers function—where each cell type's results are stored in separate Excel tabs—and generates the corresponding Gene Set Enrichment Analysis (GSEA